# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [1]:
import pandas
import re
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from scipy.stats import ttest_ind
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12,col_13,col_14
0,0,1457 m^2,1.7019,BASEBALL,eRIQ,0.121 N,Pig,569,659,1.1560,559,1052,1.0055,0.6082,-72,-0.2768
1,1,-138 m^2,0.5365,track & field,Michael,0.2781 N,rabbit,42,484,0.6524,512,166,-0.2534,0.4934,-9,0.3617
2,0,-1057 m^2,-0.6354,football,Eriq,0.4578 N,pig,1623,-582,0.8382,-571,1016,-1.8674,1.4552,1134,2.2331
3,2,1439 m^2,0.895,badminton,"Tony, Chris",-0.1466 N,dog,-324,-643,1.8613,486,1092,1.0205,-0.3142,795,-0.0169
4,0,571 m^2,0.4949,baseball,Eriq,0.4378 N,pig,-36,201,-0.4205,536,823,-0.1909,-0.2535,343,0.0196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,1,-975 m^2,0.2277,soccer,Lise,1.7978 N,rabbit,622,-597,0.0550,930,1333,-0.2782,1.737,1223,0.5306
988,3,-65 m^2,0.678,motor sports,"Reilly, Sam, Andrew",0.3491 N,hamster,NaN,1631,-0.7487,498,-2349,2.4023,0.0584,-424,-0.3207
989,0,67 m^2,1.309,basketball,Eriq,0.4343 N,pig,3037,1699,0.3501,686,25,0.4455,0.7514,241,0.4419
990,2,348 m^2,0.7354,badminton,"Lise, Tony, Chris",1.8897 N,GUINEA PIG,-470,145,0.5696,661,462,0.2797,0.2732,948,1.2869


Don't forget to checkout the column information.

In [2]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   992 non-null    int64  
 1   col_00  986 non-null    object 
 2   col_01  981 non-null    object 
 3   col_02  984 non-null    object 
 4   col_03  983 non-null    object 
 5   col_04  986 non-null    object 
 6   col_05  986 non-null    object 
 7   col_06  986 non-null    object 
 8   col_07  984 non-null    object 
 9   col_08  984 non-null    float64
 10  col_09  988 non-null    object 
 11  col_10  973 non-null    object 
 12  col_11  987 non-null    object 
 13  col_12  987 non-null    object 
 14  col_13  982 non-null    object 
 15  col_14  985 non-null    object 
dtypes: float64(1), int64(1), object(14)
memory usage: 124.1+ KB


And any numeric information.

In [3]:
unique_data.describe()

,label,col_08
count,992.00000,984.000000
mean,1.49496,0.435771
std,1.12936,0.836393
min,0.00000,-2.008900
25%,0.00000,-0.084050
50%,1.00000,0.421050
75%,3.00000,0.962375
max,3.00000,3.273700


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [4]:
def extract_numeric(val):
    if isinstance(val, str):
        match = re.search(r'[-+]?\d*\.\d+|\d+', val)
        if match:
            return float(match.group())
    return val

def is_string_column(series):
    non_null = series.dropna()
    if non_null.empty:
        return False
    converted_values = non_null.apply(lambda x: pandas.to_numeric(x, errors='coerce'))
    missing_after_conversion = converted_values.isna()
    num_not_convertible = missing_after_conversion.sum()
    return num_not_convertible / len(non_null) > 0.5

def clean_data(frame):
    clean_dataframe = frame.copy()

    for col in clean_dataframe.columns:
        non_null_values = clean_dataframe[col].dropna()

        if not non_null_values.empty:
            sample_value = non_null_values.iloc[0]
        else:
            sample_value = None
            
        if isinstance(sample_value, str) and re.search(r'\d.*[a-zA-Z]', sample_value):
            clean_dataframe[col] = clean_dataframe[col].apply(extract_numeric)

    string_cols = [col for col in clean_dataframe.columns if is_string_column(clean_dataframe[col])]
    numeric_cols = [col for col in clean_dataframe.columns if col not in string_cols]

    for col in numeric_cols:
        clean_dataframe[col] = pandas.to_numeric(clean_dataframe[col], errors='coerce')
        clean_dataframe[col] = clean_dataframe[col].fillna(clean_dataframe[col].median())
    
        if (clean_dataframe[col] % 1 == 0).all():
            clean_dataframe[col] = clean_dataframe[col].astype(int)
        else:
            clean_dataframe[col] = clean_dataframe[col].astype(float)

    for col in string_cols:
        clean_dataframe[col] = clean_dataframe[col].fillna('missing').str.lower()
        clean_dataframe[col] = clean_dataframe[col].astype(str)

    if string_cols:
        clean_dataframe = pandas.get_dummies(clean_dataframe, columns=string_cols)

    clean_dataframe.dropna(how='all', inplace=True)

    return clean_dataframe


unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_04,col_06,col_07,col_08,col_09,col_10,col_11,...,col_05_dog,col_05_ferret,col_05_fish,col_05_guinea pig,col_05_hamster,col_05_missing,col_05_pig,col_05_rabbit,col_05_rat,col_05_reptile
0,0,1457.0,1.7019,0.1210,569,659.0,1.1560,559,1052,1.0055,...,False,False,False,False,False,False,True,False,False,False
1,1,138.0,0.5365,0.2781,42,484.0,0.6524,512,166,-0.2534,...,False,False,False,False,False,False,False,True,False,False
2,0,1057.0,-0.6354,0.4578,1623,-582.0,0.8382,-571,1016,-1.8674,...,False,False,False,False,False,False,True,False,False,False
3,2,1439.0,0.8950,-0.1466,-324,-643.0,1.8613,486,1092,1.0205,...,True,False,False,False,False,False,False,False,False,False
4,0,571.0,0.4949,0.4378,-36,201.0,-0.4205,536,823,-0.1909,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,1,975.0,0.2277,1.7978,622,-597.0,0.0550,930,1333,-0.2782,...,False,False,False,False,False,False,False,True,False,False
988,3,65.0,0.6780,0.3491,660,1631.0,-0.7487,498,-2349,2.4023,...,False,False,False,False,True,False,False,False,False,False
989,0,67.0,1.3090,0.4343,3037,1699.0,0.3501,686,25,0.4455,...,False,False,False,False,False,False,True,False,False,False
990,2,348.0,0.7354,1.8897,-470,145.0,0.5696,661,462,0.2797,...,False,False,False,True,False,False,False,False,False,False


Now we should also be able to view all the numeric columns.

In [5]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992 entries, 0 to 991
Columns: 125 entries, label to col_05_reptile
dtypes: bool(112), float64(8), int64(5)
memory usage: 209.4 KB


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [6]:

def create_classifiers():
    clf1 = LogisticRegression(max_iter=5000)

    # KNN with optimized k and distance weighting
    clf2 = make_pipeline(
        StandardScaler(),
        KNeighborsClassifier(n_neighbors=7, weights='distance')
    )

    
    clf3 = DecisionTreeClassifier(
        max_depth=None,             
        min_samples_split=5,        
        min_samples_leaf=3,         
        random_state=42
    )

    return [clf1, clf2, clf3]


my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(max_iter=5000),
 Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('kneighborsclassifier',
                  KNeighborsClassifier(n_neighbors=7, weights='distance'))]),
 DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=5, random_state=42)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [7]:

def cross_fold_validation(classifier, frame, folds):
    data = frame.copy()
    X = data.drop(columns=['label'])
    y = data['label']

    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        clf = clone(classifier)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)

        acc = accuracy_score(y_test, predictions)
        scores.append(acc)

    return scores

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

Classifier: LogisticRegression, Accuracy: [0.9849246231155779, 0.9949748743718593, 0.9949494949494949, 1.0, 0.9949494949494949].
Classifier: Pipeline, Accuracy: [0.9396984924623115, 0.9547738693467337, 0.9545454545454546, 0.9191919191919192, 0.9292929292929293].
Classifier: DecisionTreeClassifier, Accuracy: [0.9195979899497487, 0.9597989949748744, 0.9444444444444444, 0.9292929292929293, 0.8939393939393939].


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [8]:
def significance_test(a_values, b_values, p_value):
    t_stat, p = ttest_ind(a_values, b_values)
    return p < p_value

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs Pipeline: True
LogisticRegression vs DecisionTreeClassifier: True
Pipeline vs DecisionTreeClassifier: False


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table needs a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.